# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


### ① 印出最新文章的「作者」「標題」「時間」

In [131]:
# 我們發現這個時間好像不是正確的，原因是我們前面存到的時間，其實不是正確的格式

# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。
# 參考圖片下載時的解法，加上 cookies 繞過驗證（後面課程會再進行補充）

import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


url = 'https://www.ptt.cc/bbs/NBA/index.html'
resp = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(resp.text, "lxml")

post_links = []
post_title_list = []
post_author_list = []
post_time_list = []

for d in soup.select(".title"):
    #print(d.select('a')[0].get('href'))

    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')

        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.select('a')[0].get('href'), cookies={'over18': '1'}).text, "lxml")
        #print('https://www.ptt.cc'+d.select('a')[0].get('href'))
        post_title_list.append(r.find_all(class_='article-meta-value')[2].text)
        #print(r.find_all(class_='article-meta-value')[2].text)
        post_author_list.append(r.find_all(class_='article-meta-value')[0].text)
        #print(r.find_all(class_='article-meta-value')[0].text)
        time = r.find_all(class_='article-meta-value')[-1].text
        post_time_list.append(datetime.strptime(time, "%a %b %d %H:%M:%S %Y"))

    except:
        continue

In [132]:
df = pd.DataFrame()
df["作者"] = post_author_list
df["標題"] = post_title_list
df["時間"] = post_time_list

index = df.sort_values("時間")[-1:].index[0]
print("最新文章的作者為 :{},\n標題為 :{},\n時間為 :{}".format(post_author_list[index],post_title_list[index],post_time_list[index]))

最新文章的作者為 :rich830502 (力取),
標題為 :[討論] 10年前的今天 The Decision,
時間為 :2020-07-09 23:11:20


### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [133]:
author_list=[]
title_list=[]
date_list=[]

url = 'https://www.ptt.cc/bbs/NBA/index.html'
resp = requests.get(url)
#print(resp.text)
soup = BeautifulSoup(resp.text, "lxml")
rnet = soup.select(".r-ent")
for r in rnet:
    for m in r.select(".meta"):
        author_list.append(m.select(".author")[0].text)
        title_list.append(r.select(".title")[0].text.strip())
        date_list.append(m.select(".date")[0].text)


df = pd.DataFrame()
df["author"]=author_list
df["title"]=title_list
df["date"]= date_list

df

,author,title,date
0,Wall62,[情報] Jamal Crawford 跟籃網簽約,7/09
1,pneumo,[情報] Kawhi Leonard因家庭因素沒隨隊去奧蘭多,7/09
2,VeryGoodBoy,[新聞] 為了季後賽拚了！馬刺波總將隨隊前往奧蘭,7/09
3,pneumo,[花邊] Josh Hart炫耀他房間裡的電玩設備,7/09
4,NCTUPARSONS,[情報] Michael Beasley準備與籃網簽約,7/09
5,pneumo,Re: [情報] Michael Beasley準備與籃網簽約,7/09
6,-,(本文已被刪除) [stark841117],7/09
7,ckny,[專欄] VC:Curry兄弟從小訓練狂，命中註定進NBA,7/09
8,ahshao,[新聞] 7人無法打復賽 籃網急簽40歲後衛,7/09
9,nO25948,[討論] 怎麼都沒有球隊考慮林書豪,7/09


### ③ 試著爬爬看其他版的文章

In [143]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

author_list=[]
title_list=[]
date_list=[]

url = 'https://www.ptt.cc/bbs/HatePolitics/index.html'
resp = requests.get(url,cookies={'over18': '1'})
#print(resp.text)
soup = BeautifulSoup(resp.text, "lxml")
rnet = soup.select(".r-ent")
for r in rnet:
    for m in r.select(".meta"):
        author_list.append(m.select(".author")[0].text)
        title_list.append(r.select(".title")[0].text.strip())
        date_list.append(m.select(".date")[0].text)


df = pd.DataFrame()
df["author"]=author_list
df["title"]=title_list
df["date"]= date_list

df

,author,title,date
0,songgood,[討論] 香港大橋能撑多久,7/09
1,mikiup0321,Re: [討論] 叫政府發現金 事後算GDP貢獻 是不是陷阱,7/09
2,songgood,[討論] 時力放話文化很嚴重嗎,7/09
3,IronCube,Re: [轉錄] 黃郁芬 FB,7/09
4,l81311i,Re: [討論] 該啟動彰化雲林合併成第七直轄市了吧！,7/09
5,dodo222kimo,Re: [新聞] 2024選總統!柯P首度明確表態:照這樣準備,7/09
6,MagicMoney,Fw: [新聞] 反送中到港版國安法 柯文哲評:台灣人看香港心驚膽顫,7/09
7,takuminauki,[新聞] 台綜院估經濟成長率1.55% 劉泰英示警股,7/09
8,vogue38,Re: [轉錄] 邱顯智：夜深了，來說說心裡話,7/10
9,victoryman,[新聞] 促高雄公民投票電子化 吳益政將聲請釋憲,7/10
